In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler
import seaborn as sns

In [4]:
!kaggle competitions download -c click-prediction-2


  0%|          | 0.00/30.8M [00:00<?, ?B/s]
  3%|3         | 1.00M/30.8M [00:00<00:09, 3.25MB/s]
  6%|6         | 2.00M/30.8M [00:00<00:08, 3.37MB/s]
 10%|9         | 3.00M/30.8M [00:00<00:08, 3.43MB/s]
 13%|#2        | 4.00M/30.8M [00:01<00:08, 3.46MB/s]
 16%|#6        | 5.00M/30.8M [00:01<00:07, 3.48MB/s]
 19%|#9        | 6.00M/30.8M [00:01<00:07, 3.49MB/s]
 23%|##2       | 7.00M/30.8M [00:02<00:07, 3.50MB/s]
 26%|##5       | 8.00M/30.8M [00:02<00:06, 3.51MB/s]
 29%|##9       | 9.00M/30.8M [00:02<00:06, 3.50MB/s]
 32%|###2      | 10.0M/30.8M [00:03<00:06, 3.51MB/s]
 36%|###5      | 11.0M/30.8M [00:03<00:05, 3.51MB/s]
 39%|###8      | 12.0M/30.8M [00:03<00:05, 3.51MB/s]
 42%|####2     | 13.0M/30.8M [00:03<00:05, 3.52MB/s]
 45%|####5     | 14.0M/30.8M [00:04<00:05, 3.16MB/s]
 49%|####8     | 15.0M/30.8M [00:04<00:05, 3.26MB/s]
 52%|#####1    | 16.0M/30.8M [00:04<00:04, 3.33MB/s]
 55%|#####5    | 17.0M/30.8M [00:05<00:04, 3.38MB/s]
 58%|#####8    | 18.0M/30.8M [00:05<00:03, 3.42MB/s]
 

In [6]:
pd.read_csv("out ilya special.csv").click.value_counts()

0    98865
1      135
Name: click, dtype: int64

In [4]:
train = pd.read_csv("./click-prediction-2/train.csv")
test = pd.read_csv("./click-prediction-2/test.csv")

In [5]:
X_train, y_train = train.drop(["click"], axis=1), train["click"]
X_test = test

In [623]:
X_train, X_test, y_train, y_test = train_test_split(train, train["click"], test_size=.25, shuffle=True, random_state=1)

In [56]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201000 entries, 0 to 200999
Data columns (total 40 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   row_number            201000 non-null  int64  
 1   ssp_price             201000 non-null  float64
 2   show_ts               201000 non-null  int64  
 3   show_reason           201000 non-null  int64  
 4   billable              201000 non-null  int64  
 5   stat_id               201000 non-null  int64  
 6   session_id            195640 non-null  object 
 7   plan                  201000 non-null  int64  
 8   height                201000 non-null  int64  
 9   thematics             201000 non-null  object 
 10  instl                 201000 non-null  int64  
 11  dmp_commission        201000 non-null  object 
 12  ssp_name              201000 non-null  object 
 13  sub_plan              184794 non-null  float64
 14  width                 201000 non-null  int64  
 15  

In [57]:
o_session_id = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-10).fit(np.array(X_train.session_id.fillna("-1")).reshape(-1, 1))
new_session_id = o_session_id.transform(np.array(X_train.session_id.fillna("-1")).reshape(-1, 1))

o_bidid = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-10).fit(np.array(X_train.bidid).reshape(-1, 1))
new_bidid = o_bidid.transform(np.array(X_train.bidid).reshape(-1, 1))

o_bid_id = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-10).fit(np.array(X_train.bid_id).reshape(-1, 1))
new_bid_id = o_bid_id.transform(np.array(X_train.bid_id).reshape(-1, 1))

o_sync_session_status = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-10).fit(np.array(X_train["sync_session_status"]).reshape(-1, 1))
new_sync_session_status = o_sync_session_status.transform(np.array(X_train.sync_session_status).reshape(-1, 1))

o_dmp_commission = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-10).fit(np.array(X_train["dmp_commission"].fillna("-1")).reshape(-1, 1))
dmp_commission = o_dmp_commission.transform(np.array(X_train["dmp_commission"].fillna("-1")).reshape(-1, 1))

In [58]:
new_session_id_test = o_session_id.transform(np.array(X_test.session_id.fillna("-1")).reshape(-1, 1))
new_bidid_test = o_bidid.transform(np.array(X_test.bidid).reshape(-1, 1))
new_bid_id_test = o_bid_id.transform(np.array(X_test.bid_id).reshape(-1, 1))
new_sync_session_status_test = o_sync_session_status.transform(np.array(X_test.sync_session_status).reshape(-1, 1))
dmp_commission_test = o_dmp_commission.transform(np.array(X_test["dmp_commission"].fillna("-1")).reshape(-1, 1))

In [59]:
arr = []
for x in X_train.columns:
    if "_id" in x and str(train[x].dtype) != "int64":
        arr.append(x)
        
arr = []
for x in X_test.columns:
    if "_id" in x and str(train[x].dtype) != "int64":
        arr.append(x)

In [60]:
def clear(x):
    if x != "[]":
        x = [int(x) for x in x[1:-1].split(", ")]
        return x
    else:
        return [0]

In [61]:
thematics_train = X_train["thematics"].apply(clear)
thematics_test = X_test["thematics"].apply(clear)

In [62]:
thematics_dummies_train = pd.get_dummies(thematics_train.apply(pd.Series).stack()).groupby(level=0).sum()
thematics_dummies_test = pd.get_dummies(thematics_test.apply(pd.Series).stack()).groupby(level=0).sum()

In [63]:
thematics_dummies_train.columns = thematics_dummies_train.columns.astype(np.int)
thematics_dummies_test.columns = thematics_dummies_test.columns.astype(np.int)

In [64]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE

pca = TruncatedSVD(n_components=7, random_state=1).fit(thematics_dummies_train)
thematics_dummies_svd_train = pca.transform(thematics_dummies_train)

In [13]:
set(thematics_dummies_train.columns) - set(thematics_dummies_test.columns)

set()

In [633]:
thematics_dummies_test[61] = 0

In [65]:
thematics_dummies_svd_test = pca.transform(thematics_dummies_test)

In [66]:
i1 = 7
X = X_train.drop(arr + ["dmp_commission"] + ["ssp_name"] + ["bidid"] + ["sync_session_status"] + ["thematics"] + ["ssp_id"] + ["row_number"], axis=1)

X_new = pd.concat([X, pd.DataFrame(thematics_dummies_svd_train, columns=list(range(i1))), pd.DataFrame(dmp_commission.flatten(), columns=["dmp_commission"]),
                   pd.DataFrame(new_sync_session_status.flatten(), columns=["sync_session_status"]), 
                   pd.DataFrame(new_session_id.flatten(), columns=["session_id"]), 
                   pd.DataFrame(new_bidid.flatten(), columns=["bidid"])], axis=1)
X_new['sub_plan'] = X_new['sub_plan'].fillna(X_new['sub_plan'].mean())
X_new['sub_quantity'] = X_new['sub_quantity'].fillna(X_new['sub_quantity'].mean())

In [67]:
X = X_test.drop(arr + ["dmp_commission"] + ["ssp_name"] + ["bidid"] + ["sync_session_status"] + ["thematics"] + ["ssp_id"] + ["row_number"], axis=1)

X_new_test = pd.concat([X, pd.DataFrame(thematics_dummies_svd_test, columns=list(range(i1))), pd.DataFrame(dmp_commission_test.flatten(), columns=["dmp_commission"]), 
                   pd.DataFrame(new_sync_session_status_test.flatten(), columns=["sync_session_status"]), 
                   pd.DataFrame(new_session_id_test.flatten(), columns=["session_id"]), 
                   pd.DataFrame(new_bidid_test.flatten(), columns=["bidid"])], axis=1)
X_new_test['sub_plan'] = X_new_test['sub_plan'].fillna(X_new_test['sub_plan'].mean())
X_new_test['sub_quantity'] = X_new_test['sub_quantity'].fillna(X_new_test['sub_quantity'].mean())

In [17]:
X_new.to_csv("X_train_clean.csv", index=False)
X_new_test.to_csv("X_train_clean_test.csv", index=False)

In [3]:
X_new = pd.read_csv("X_train_clean.csv")
X_new_test = pd.read_csv("X_train_clean_test.csv")

In [709]:
sum(np.round(np.sort(rfc.feature_importances_), 4) == 0)

61

In [74]:
len(X_new.columns[np.argsort(rfc.feature_importances_)])

39

In [68]:
rfc = RandomForestClassifier(n_estimators=500, n_jobs=-1, oob_score=True, random_state=1).fit(X_new, y_train)

In [69]:
rfc.oob_score_

0.9957960199004975

0.9957960199004975 - svd -7 opt  

In [ ]:
np.array([1, 4, 5, 10, 14, 15, 17, 19, 21, 22, 24, 26]) - 1

Index(['ssp_price', 'billable', 'stat_id', 'width', 'ssp_bid', 'sub_quantity',
       'ssp_site_id', 'campaign_id', 'bid_ts', 'flight_id', 'quantity', 'bid'],
      dtype='object')

In [ ]:
1 4 5 10 14 15 17 19 21(589) 22 24(573) 26(607)

In [90]:
np.argsort(rfc.feature_importances_)[~(np.array([1, 4, 5, 10, 14, 15, 17, 19, 21, 22, 24, 26]) - 1)]

array([23, 14,  8, 20, 16,  2, 11, 18, 25,  7,  3, 12], dtype=int64)

In [ ]:
rfc2 = RandomForestClassifier(n_estimators=500, n_jobs=-1, oob_score=True, random_state=1).\
fit(X_new.drop(X_new.columns[np.argsort(rfc.feature_importances_)[~(np.array([1, 4, 5, 10, 14, 15, 17, 19, 21, 22, 24, 26]) - 1)]], axis=1), y_train)
rfc2.oob_score_

0.9961094527363185

In [77]:
arr = []
for i in range(27):
    rfc1 = RandomForestClassifier(n_estimators=500, n_jobs=-1, oob_score=True, random_state=1).fit(X_new.drop(X_new.columns[np.argsort(rfc.feature_importances_)][:i], axis=1), y_train)
    arr.append([rfc1, rfc1.oob_score_])
    print(rfc1.oob_score_)

0.9957960199004975
0.9957611940298507
0.995776119402985
0.9958009950248756
0.9957512437810945
0.995726368159204
0.9957611940298507
0.9957611940298507
0.9957661691542289
0.9958159203980099
0.9958059701492538
0.9958109452736318
0.9958109452736318
0.9958606965174129
0.9958358208955224
0.9957910447761193
0.9959253731343284
0.9958855721393035
0.9959104477611941
0.9958656716417911
0.9959203980099502
0.9958955223880597
0.9958656716417911
0.9958855721393035
0.9957363184079602
0.9960796019900497
0.9960796019900497


0.996457711442786


In [ ]:
for (i, x) in enumerate(zip(np.sort(rfc.feature_importances_), X_new.columns[np.argsort(rfc.feature_importances_)])):
    print(i, x)

0 (4.311568113964588e-08, 'sync_session_status')
1 (0.00011229944718803416, 'adblock')
2 (0.00012258951191887916, 'bid_type')
3 (0.0005172547379843234, 'width')
4 (0.0006328014930621334, 0)
5 (0.003571110242985077, 6)
6 (0.0036807028846386615, 5)
7 (0.003802229584569241, 4)
8 (0.004146853205004815, 2)
9 (0.004207094454953624, 3)
10 (0.004578722159534966, 1)
11 (0.004669481810100277, 'dmp_commission')
12 (0.005787896919581619, 'height')
13 (0.008498787468079763, 'device_type')
14 (0.009224336325343013, 'sales_category_id')
15 (0.020031034654569232, 'account_id')
16 (0.025233267253091097, 'flight_id')
17 (0.026748566929476695, 'creative2flight_id')
18 (0.043423542764439514, 'tagid')
19 (0.04637587039931236, 'stat_id')
20 (0.06716097235824206, 'ssp_price')
21 (0.11135840722159346, 'request_ts')
22 (0.11288703206964573, 'show_ts')
23 (0.11548900121508983, 'session_id')
24 (0.12377038289542756, 'ssp_bid')
25 (0.12612650130362657, 'plan')
26 (0.1278432175748604, 'bidid')


In [224]:
for (i, x) in enumerate(zip(np.sort(rfc1.feature_importances_), X_sq_train.columns[np.argsort(rfc1.feature_importances_)])):
    print(i, x)

0 (0.027040779753104954, '0P')
1 (0.030525673371642754, '2P')
2 (0.031157643603126717, 'stat_id')
3 (0.032382706411283525, '1P')
4 (0.03416417334590678, '3P')
5 (0.034660002910383084, '4P')
6 (0.0415615772780319, 'ssp_price')
7 (0.06445821274276432, 'bid_ts')
8 (0.06461240408355585, 'request_ts')
9 (0.06551085814198729, 'show_ts')
10 (0.07512823657568855, 'sub_plan')
11 (0.0792912217513335, 'session_id')
12 (0.0809273554619468, 'plan')
13 (0.0812528847736526, 'sub_quantity')
14 (0.08323044651656801, 'ssp_bid')
15 (0.08480335108511086, 'quantity')
16 (0.08929247219391247, 'bidid')


In [102]:
X_new_1 = X_new.drop(X_new.columns[np.argsort(rfc.feature_importances_)[~(np.array([1, 4, 5, 10, 14, 15, 17, 19, 21, 22, 24, 26]) - 1)]], axis=1)

In [106]:
from sklearn.decomposition import PCA, TruncatedSVD
i2 = 12
i3 = 3
columns = [x for x in X_new_1.columns[np.argsort(rfc2.feature_importances_)]][:i2]
svd_drop = TruncatedSVD(n_components=i3, random_state=1).fit(X_new_1[columns])
wh_train = pd.DataFrame(svd_drop.transform(X_new_1[columns]), columns=[str(x) + 'P' for x in range(i3)])

X_sq_train = pd.concat([X_new.drop(columns, axis=1), wh_train], axis=1)

rfc1 = RandomForestClassifier(n_estimators=500, n_jobs=-1, oob_score=True, random_state=1).fit(X_sq_train, y_train)
rfc1.oob_score_

0.9957960199004975

0.9958009950248756

0.9963781094527363 - 25 - 5 SVD; baseline  
0.996457711442786 - 27 - 5 SVD; baseline?  

In [483]:
wh_test = pd.DataFrame(svd_drop.transform(X_new_test[columns]), columns=[str(x) + 'P' for x in range(i3)])
X_sq_test = pd.concat([X_new_test.drop(columns, axis=1), wh_test], axis=1)

In [66]:
i5 = 3
columns = ["show_ts", "bid_ts", "request_ts"]
svd_drop = PCA(n_components=i5, random_state=1).fit(X_sq_train[columns])
wh_train = pd.DataFrame(svd_drop.transform(X_sq_train[columns]), columns=[str(x) + 'P' for x in range(i5)])
X_sq_train_pca = pd.concat([X_sq_train.drop(columns, axis=1), wh_train], axis=1)

In [67]:
rfc2 = RandomForestClassifier(n_estimators=500, n_jobs=-1, oob_score=True, random_state=1).fit(X_sq_train_pca, y_train)
rfc2.oob_score_

0.9956865671641791

show_ts, request_ts, bid_ts - PCA doesn't work  
"plan", "sub_plan" - PCA -

In [234]:
y_out_pred = rfc1.predict(X_sq_test)

In [841]:
y_out.click.value_counts()

0    98841
1      159
Name: click, dtype: int64

In [235]:
y_out = pd.concat([test["row_number"], pd.Series(y_out_pred)], axis=1)
y_out.columns = ["row_number", "click"]
y_out.to_csv("out.csv", index=False)

In [712]:
rfc1 = RandomForestClassifier(n_estimators=500, n_jobs=-1, oob_score=True, random_state=1).fit(X_new1, y_train)
rfc1.oob_score_

0.9952769485903814

In [714]:
log_loss(y_test, rfc1.predict(X_new_test1))

0.1732094179246249

0.18558148510698372
0.9948192371475953
0.99505
0.9950315091210613 - 7
0.9950845771144279 - 4
0.9950912106135986 - 2
0.17183477208193912 - 1
0.1752714344258136 - 2
0.1855 - 70
0.995469320066335 - theme, bid_t
0.16839809382567797 - 
0.9948789386401327 - stock
0.1848941462732542 - stock
0.9954626865671642 - adb
0.9954958540630182 - width
0.9954825870646766 - adb width
0.9955223880597015 - dmp
0.9954759535655058
0.9951442786069652 - all theme and feature (not bid_id)
0.9952305140961858 - 30 drop
0.9951310116086235 - 10 drop
0.9951442786069652
0.9953897180762853 - PCA 10 components
0.9954295190713102 - PCA 3 comp
0.9957412935323383
0.9957014925373134
0.9957512437810945
0.995776119402985 - SVD 5 comp
0.9957960199004975 - SVD 7 comp
0.995731343283582 - SVD 11 comp
0.9957363184079602
0.995731343283582
0.9957960199004975

In [81]:
test = pd.read_csv("./click-prediction-2/test.csv")

In [82]:
arr = []
for x in test.columns:
    if "_id" in x and str(train[x].dtype) != "int64":
        arr.append(x)

In [128]:
X_test = test.drop(arr, axis=1)

In [84]:
dmp_commission_test = X_test["dmp_commission"].apply(dmp)

In [85]:
thematics_test = X_test["thematics"].apply(clear)
thematics.head(10)

0                      [-1]
1                      [-1]
2                      [-1]
3                      [-1]
4                      [-1]
5                      [-1]
6                      [-1]
7                      [-1]
8    [90, 109, 102, 78, 88]
9                      [-1]
Name: thematics, dtype: object

In [86]:
thematics_dummies_test = pd.get_dummies(thematics_test.apply(pd.Series).stack()).groupby(level=0).sum()

In [87]:
thematics_dummies_test.columns = thematics_dummies_test.columns.astype(np.int)

In [88]:
sync_session_status_test = X_test["sync_session_status"]

In [127]:
new_session_id_test = o_session_id.transform(np.array(test.session_id.fillna("-1")).reshape(-1, 1))
new_bid_id_test = o_bid_id.transform(np.array(test.bid_id.fillna("-1")).reshape(-1, 1))
new_bidid_test = o_bidid.transform(np.array(test.bidid.fillna("-1")).reshape(-1, 1))

In [189]:
X_test = test.drop(arr + ["dmp_commission"] + ["ssp_name"] + ["bidid"] + ["sync_session_status"] + ["thematics"] + ["row_number"], axis=1)

In [130]:
X_new_test = pd.concat([X_test, thematics_dummies_test, dmp_commission_test, sync_session_status_test, pd.DataFrame(new_session_id_test, columns=["session_id"]), 
                       pd.DataFrame(new_bid_id_test, columns=["bid_id"]), 
                       pd.DataFrame(new_bidid_test, columns=["bidid"])], axis=1)

In [138]:
y_out.click.value_counts()

0    98918
1       82
Name: click, dtype: int64

In [133]:
y_test = rfc.predict(X_new_test)

In [132]:
X_new_test['sub_quantity'] = X_new_test['sub_quantity'].fillna(-1)
X_new_test['sub_plan'] = X_new_test['sub_plan'].fillna(-1)

In [136]:
y_out = pd.concat([test["row_number"], pd.Series(y_test)], axis=1)

In [137]:
y_out.columns = ["row_number", "click"]

In [139]:
y_out.to_csv("out.csv", index=False)

In [12]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, SCORERS, log_loss, mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostClassifier
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import RobustScaler, StandardScaler
from catboost import CatBoostClassifier

In [ ]:
X_robust = StandardScaler().fit_transform(X_sq_train)
X_train_r, X_valid, y_train_r, y_valid = train_test_split(X_robust, y_train, random_state=537, test_size=.20, shuffle=True)
X_train_n, X_test, y_train_n, y_test = train_test_split(X_train_r, y_train_r, random_state=537, test_size=.20, shuffle=True)

In [ ]:
robust_scaler = StandardScaler().fit(X_sq_train)
X_scaler_train = robust_scaler.transform(X_sq_train)
X_scaler_test = robust_scaler.transform(X_sq_test)   

NameError: name 'X_sq_train' is not defined

In [494]:
for i, x in enumerate(zip(np.sort(a3.feature_importances_), X_sq_train.columns[np.argsort(a3.feature_importances_)])):
    print(i, x)

0 (0.0, 'bid_ts')
1 (0.038044494, 'ssp_price')
2 (0.04839327, '0P')
3 (0.050223734, 'sub_plan')
4 (0.050976172, 'plan')
5 (0.051142637, 'ssp_bid')
6 (0.052389838, 'quantity')
7 (0.053821743, 'show_ts')
8 (0.054753873, '4P')
9 (0.055445526, '2P')
10 (0.056014016, 'stat_id')
11 (0.056399263, 'session_id')
12 (0.05783613, 'bidid')
13 (0.057997372, 'sub_quantity')
14 (0.06229765, 'request_ts')
15 (0.09890033, '3P')
16 (0.15536396, '1P')


In [479]:
a4 = xgb.XGBClassifier(alpha=.4, n_estimators=350, random_state=42, max_depth=20, tree_method='gpu_hist', gpu_id=0, learning_rate=.02, use_label_encoder=False, eval_metric="logloss")
-np.mean(cross_val_score(a4, pd.DataFrame(X_robust).drop(np.argsort(a3.feature_importances_)[:4], axis=1), y_train, cv=5, scoring="neg_log_loss"))

0.02439208797227263

0.02506480851613329  
0.02438125480368151 :3
0.024370819273096943 :4

In [495]:
a5 = xgb.XGBClassifier(alpha=.4, n_estimators=350, random_state=42, max_depth=20, tree_method='gpu_hist', gpu_id=0, learning_rate=.02, 
                       use_label_encoder=False, eval_metric="logloss").fit(pd.DataFrame(X_scaler_train).drop(np.argsort(a3.feature_importances_)[:4], axis=1), y_train)


In [ ]:
a3 = xgb.XGBClassifier(alpha=.4, n_estimators=350, random_state=42, max_depth=20, tree_method='gpu_hist', gpu_id=0, learning_rate=.02, 
                       use_label_encoder=False, eval_metric="logloss").fit(X_scaler_train, y_train)

KeyboardInterrupt: 

0.027616484837554384 - cv - max_depth=6, n_estimators=500, learning_rate=.1, random_state=42, use_label_encoder=False, tree_method='gpu_hist', gpu_id=0, eval_metric="logloss"  
0.02581962736736757 - cv - alpha=.4, n_estimators=350, random_state=42, max_depth=20, tree_method='gpu_hist', gpu_id=0, learning_rate=.02, use_label_encoder=False, eval_metric="logloss"  
0.025743451440242492:0.026307132926935196  
ss cv - alpha=.4, n_estimators=350, random_state=42, max_depth=20, tree_method='gpu_hist', gpu_id=0, learning_rate=.02, use_label_encoder=False, eval_metric="logloss"  
277

In [513]:
X_scaler_train_4 = pd.DataFrame(X_scaler_train).drop(np.argsort(a3.feature_importances_)[:4], axis=1)
X_scaler_test_4 = pd.DataFrame(X_scaler_test).drop(np.argsort(a3.feature_importances_)[:4], axis=1)

In [514]:
X_scaler_train_4.to_csv("X_scaler_train_4.csv")
X_scaler_test_4.to_csv("X_scaler_test_4.csv")

In [3]:
X_scaler_train_4 = pd.read_csv("X_scaler_train_4.csv")
X_scaler_test_4 = pd.read_csv("X_scaler_test_4.csv")

In [10]:
rs = StandardScaler().fit(X_scaler_train_4)
X_scaler_train_4_r = rs.transform(X_scaler_train_4)
X_scaler_test_4_r = rs.transform(X_scaler_test_4)

X_train_r, X_valid, y_train_r, y_valid = train_test_split(X_scaler_train_4_r, y_train, random_state=537, test_size=.20, shuffle=True)
X_train_n, X_test, y_train_n, y_test = train_test_split(X_train_r, y_train_r, random_state=537, test_size=.20, shuffle=True)

In [25]:
c1 = CatBoostClassifier(task_type="GPU", loss_function="Logloss", random_state=42, max_depth=16).fit(X_train_n, y_train_n)

Learning rate set to 0.025808
0:	learn: 0.6244037	total: 80.2ms	remaining: 1m 20s
1:	learn: 0.5617486	total: 686ms	remaining: 5m 42s
2:	learn: 0.5069019	total: 750ms	remaining: 4m 9s
3:	learn: 0.4574675	total: 1.29s	remaining: 5m 20s
4:	learn: 0.4150726	total: 1.83s	remaining: 6m 5s
5:	learn: 0.3766381	total: 2.38s	remaining: 6m 33s
6:	learn: 0.3418643	total: 2.92s	remaining: 6m 54s
7:	learn: 0.3104345	total: 3.46s	remaining: 7m 9s
8:	learn: 0.2820851	total: 4s	remaining: 7m 20s
9:	learn: 0.2578882	total: 4.55s	remaining: 7m 30s
10:	learn: 0.2355888	total: 5.11s	remaining: 7m 39s
11:	learn: 0.2156173	total: 5.65s	remaining: 7m 45s
12:	learn: 0.1986001	total: 6.2s	remaining: 7m 50s
13:	learn: 0.1831073	total: 6.74s	remaining: 7m 54s
14:	learn: 0.1693967	total: 7.29s	remaining: 7m 58s
15:	learn: 0.1558139	total: 7.83s	remaining: 8m 1s
16:	learn: 0.1442359	total: 8.37s	remaining: 8m 4s
17:	learn: 0.1340123	total: 8.91s	remaining: 8m 6s
18:	learn: 0.1246529	total: 9.45s	remaining: 8m 7s
19

In [26]:
log_loss(y_valid, c1.predict_proba(X_valid))

0.031219132375522735

0.0357329855587518  
0.035666214437294716 C - 1.1

In [503]:
X_train_r, X_valid, y_train_r, y_valid = train_test_split(X_scaler_train_4, y_train, random_state=537, test_size=.20, shuffle=True)

In [ ]:
log_loss(y_valid, a2.predict(X_valid))

0.0335046216536139

In [ ]:
log_loss(y_test, a3.predict_proba(pd.DataFrame(X_test).drop(np.argsort(a3.feature_importances_)[:4], axis=1))[:, 1])

0.03257437668647646

In [ ]:
y_out = a5.predict_proba(pd.DataFrame(X_scaler_test).drop(np.argsort(a3.feature_importances_)[:4], axis=1))[:, 1]

In [ ]:
y_out = pd.concat([test["row_number"], pd.Series(y_out)], axis=1)
y_out.columns = ["row_number", "click"]

In [ ]:
y_out.to_csv("out_a2.csv", index=False)

In [ ]:
from sklearn.svm import SVC, SVR

In [ ]:
svc = SVR().fit(X_train_n, y_train_n)

In [500]:
log_loss(y_train, np.zeros(201000) + .009999)

0.03583743090715849

In [441]:
pd.Series(np.round(svc.predict(X_valid)).astype(int)).unique()

array([0])

In [444]:
y_valid.value_counts()

0    19967
1      133
Name: click, dtype: int64

In [482]:
y_train

0         0
1         0
2         0
3         0
4         0
         ..
200995    0
200996    0
200997    0
200998    0
200999    0
Name: click, Length: 201000, dtype: int64

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [466]:
y_valid.values

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [467]:
np.zeros(200100).astype(int)

array([0, 0, 0, ..., 0, 0, 0])

In [471]:
log_loss

<function sklearn.metrics._classification.log_loss(y_true, y_pred, *, eps=1e-15, normalize=True, sample_weight=None, labels=None)>